<div class='alert alert-info'>
<text style='color:black;'>
<b>Послание о ноутбуке</b>
<p>
1 В начале была загрузка всех необходимых библиотек и модулей.

2 Затем явилась переменная `conn`, которая отвечала за соединение с исходной базой данных.

3 И переменная эта была типа `sqlalchemy.Connection`, который несколько отличается от похожего по названию типа `sqlite3.Connection`. Ясней начертано в разделе "Нюансы".

4 И была вскоре предварительно обработана база данных с помощью функции `prepare_column_names`, которая переименовывала все столбцы и таблицы с нерадивыми названиями.

5 И стала переменная `queries` типа `IterableDataFrame`, отвечающая за таблицу с вопросами и ответами

6 И вот господствующий класс `HuggingFaceModelInference`, который всему был свет. Его главным методом был метод `evaluate`, который и нёс всю суть. 

7 Метод этот вначале подгружал указанную в конструкторе модель.

8 И позже начинался порочный цикл, в котором являлось чадо с названием `builder`. Чадо это возводило промпт, исходя из базы данных `connection` и перечисленных фичей. 

9 И в чёртовом котле варились переменные `input` и `output`, которые были входными токенами модели и сгенерированным ответом соответственно.

10 И пыталась переменная `pred_sql` регулярным ковшом вытянуть оттуда сгенерированный самородок.

11 И сравнивались вскоре переменные `pred_sql` и `gold_sql`, выясняя, является ли самородок подлинным или нет.

12 Так и заканчивался порочный цикл.

13 Обрабатывал метод этот ряд моделей, в число которого вошли SQLCoder, Deepseek, ChatDB и DuckDB.
</p>
</text>
</div>

<div class='alert alert-danger'>
<text style='color:black;'>
<b>Косяки:</b>

1. Schema linking не распознает спецификатор *
2. Сравнение таблиц не работает достаточно гибко для датафреймов с разными количествами столбцов
3. Использование регулярного выражения для вытягивания ответа модели несёт риск потери информации. Такой подход неустойчив
4. Не осуществлен автоматический перебор вариантов для `build_prompt`
</text>
</div>

<div class='alert alert-warning'>
<text style='color:black;'>
<b>Нюансы:</b>

1. Для тестирования моделей необходимы два объекта: непосредственно соединение с базой данных и таблица с запросами к этим данным.
Таблица с запросами должна удовлетворять одному условию - она должна состоять из столбцов с названиями 'question' и 'query'.
К базе данных строгих требований нет.

2. Существуют, по крайней мере, два модуля в питоне, которые предоставляют интерфейс взаимодействия с базами данных SQlite -- sqlite3 и sqlalchemy. 
Мы будем пользоваться модулем sqlalchemy по той простой причине, что он позволяет напрямую читать .xlsx таблицы как SQlite базу данных. Важно, что в библиотеке
sqlite3, чтобы сделать запрос в бд, надо написать строку вида `conn.execute(query)`, где query - str. В sqlalchemy немного иначе - `conn.execute(text(query))`;
функция text лежит в этом же модуле. 
</text>
</div>

## Загрузка необходимых модулей и датасета

In [ ]:
!pip install json5 gdown sentence-transformers -q

In [1]:
import json5
from tqdm import tqdm
import pandas as pd
import re, time
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModelForCausalLM
#import numpy as np
#from utils.general import *
#import torch

from tree_edit_distance import SqlNode, ratio, parse_sql
from sqlalchemy import create_engine
from prompting import *
from sklearn.utils import shuffle
from sqlalchemy import text, Connection
from utils.dataset import prepare_column_names, structure_from_connection, IterableDataFrame
import matplotlib.pyplot as plt

In [ ]:
#!gdown 1Xjbp207zfCaBxhPgt-STB_RxwNo2TIW2
#unzip_file('merged_database_2022-06-10.zip', 'pauq_databases')

In [2]:
engine = create_engine('sqlite:///main_database.sqlite', echo=False)
conn = engine.connect()

In [3]:
prepare_column_names(conn) # Устраняет пробелы в названии столбцов
queries = IterableDataFrame(pd.read_excel('NLSQL.xlsx'))

In [ ]:
# import itertools

# def recove_table(table : pd.DataFrame, subtable_structure : dict):
#         pieces = []
#         for col in subtable_structure['columns']:
#                 if col in table.columns:
#                         pieces.append(table[col])
#                 else:
#                         pieces.append(pd.DataFrame({col : [None] * table.shape[0]}))

#         recovered_table = pd.concat(pieces, axis=1)
#         return recovered_table

# def SFC(table1 : pd.DataFrame, table2 : pd.DataFrame, subtable_structure : dict):
#         foreign_col1 = set(table1.columns) ^ set(subtable_structure['columns'])
#         foreign_col2 = set(table2.columns) ^ set(subtable_structure['columns'])
        
#         if len(foreign_col1) != len(foreign_col2):
#                 return 0.0

#         right_table1 = recove_table(table1, subtable_structure)
#         right_table2 = recove_table(table2, subtable_structure)

#         permutations = list(itertools.permutations([i for i in range(len(foreign_col1))]))
        

# Препроцессинг промпта

In [ ]:
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
class HuggingFaceModelInference:
    def __init__(self, path):
        self.path = path
        self.evaluated = False
        self.is_downloaded = False


    def __load_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
                    self.path,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    max_memory={0: "10GiB", 1: "10GiB"},  
                    offload_folder="./offload", 
                    trust_remote_code=True
                    )


    def evaluate(self, queries : IterableDataFrame, connection : Connection):
        if not self.is_downloaded:
            self.__load_model()
            self.is_downloaded = True


        self.model.eval()
        logger = []
        summary = 0
        for query in tqdm(queries):
            question = query['question']
            gold_sql = query['query']

            builder = PromptBuilder(question)
            prompt = builder.add_schema_template(connection)\
                             .build_prompt(4)

            text = f'''You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
                1. Return ONLY valid SQL query without any explanations
                3. Never repeat the answer
                4. Format: [SQL]<query>[/SQL]
                
                ### Instruction:
                {prompt}\n\n
                ### Response:'''
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            with torch.inference_mode():  
                inputs = self.tokenizer(text,return_tensors="pt").to(self.model.device) 

                generate_ids = self.model.generate(
                                **inputs,
                                max_length=2048,
                                num_return_sequences=1,
                                temperature=0.2, 
                                top_p=0.95,
                                do_sample=True,
                                use_cache=True 
                                )
        
                output = self.tokenizer.decode(
                        generate_ids[0, inputs.input_ids.shape[1]:],
                        skip_special_tokens=True
                        )

            #pred_sql = re.search(r'Response:(.+)', output, re.DOTALL).group(1).strip()
            pred_sql =  re.search(r'\[SQL\](.*?)\[\/SQL\]', output, re.DOTALL)
            pred_sql = pred_sql.group(1).strip() if pred_sql else "error"
            logger.append({'question' : question, 'pred' : pred_sql, 'gold' : gold_sql})
            
            try:
                df_pred = pd.read_sql(pred_sql, connection)
                df_gold = pd.read_sql(gold_sql, connection)
                summary += table_similarity(df_pred, df_gold, mode='flexible')
            except:
                pass

        self.summary = summary
        self.queries_count = len(queries)
        self.logger = logger
        self.evaluated = True


    def accuracy(self):
        """
        Значение метрики Accuracy для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        return self.summary / self.queries_count


    def TED(self):
        """
        Значение метрики Tree Edit Distance для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        summary = 0
        for sample in self.logger:
            try:
                pred_root = parse_sql(sample['pred'])
                gold_root = parse_sql(sample['gold'])
                summary += ratio(pred_root, gold_root)
            except:
                pass

        return summary / self.queries_count

In [ ]:
def dump_inference(name: str, exec_time: list, sql_sim, acc):
    dump = json5.dumps({
        'name': name,
        'exec_time': exec_time,
        'sql_similarity': str(sql_sim),
        'accuracy': str(acc)
    })
    with open(f'{name}_dump.txt', 'w') as w:
        w.write(dump)

## 1. SQLCoder 7b https://huggingface.co/defog/sqlcoder-7b-2

In [ ]:
sqlcoder = HuggingFaceModelInference('defog/sqlcoder-7b-2')
sqlcoder.evaluate(shuffle(dataset))

In [ ]:
sqlcoder.accuracy(), sqlcoder.sql_similarity(), np.mean(sqlcoder.exec_time)

In [ ]:
sqlcoder.logger

## DeepSeek coder 6.7b

In [ ]:
deepseek = HuggingFaceModelInference('deepseek-ai/deepseek-coder-6.7b-instruct')
deepseek.evaluate(shuffle(queries.as_list())[:10], conn) 

## 3. Chat2DB 7b

In [ ]:
# chat2db = HuggingFaceModelInference('Chat2DB/Chat2DB-SQL-7B')
# chat2db.evaluate(shuffle(dataset)[:20])

In [ ]:
# chat2db.accuracy(), chat2db.sql_similarity(), np.mean(chat2db.exec_time)

In [ ]:
# dump_inference('Chat2DB-SQL-7B', chat2db.exec_time, chat2db.sql_similarity(), chat2db.accuracy())

## 5. DuckDB-NSQL 7b

In [ ]:
# duckdb = HuggingFaceModelInference('motherduckdb/DuckDB-NSQL-7B-v0.1')

In [ ]:
# duckdb.evaluate(shuffle(dataset)[:30])

In [ ]:
# duckdb.accuracy(), duckdb.sql_similarity(), np.mean(duckdb.exec_time)

## Прочее

In [ ]:
from numba import cuda
import gc
cuda.devices.gpus[0].reset()
cuda.devices.gpus[1].reset()
gc.collect()